In [31]:
import matplotlib.pyplot as plt
import nltk
import re
import spacy
nltk.download('gutenberg')
# benchmark data
# https://spacy.io/usage/facts-figures#benchmarks

# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_sm")

# Add neural coref to SpaCy's pipe
# https://github.com/huggingface/neuralcoref
import neuralcoref
neuralcoref.add_to_pipe(nlp)

from pyvis.network import Network
# import networkx as nx
import itertools
import math
import numpy as np

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/naimulhoque/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [63]:
#book = nltk.corpus.gutenberg.raw('carroll-alice.txt')
#doc = nlp(book)

# file_name = '../static/data/story/birthright.txt' #
file_name = 'HP - The Philosopher Stone.txt' #
book = open(file_name, encoding="utf8").read()
book = re.sub(r'\W*Page |.*J.K. Rowling\W*', ' ', book)

In [35]:
def delimiter(txt, gran="para"):
    limits = []
    
    if gran=="para":
        for i, m in enumerate(re.finditer('\n\n', txt)):
            if i>0 and i%1==0:  # consider 10 paras as single entity
                limits.append(m.start())
    elif gran=="CHAPTER":
        limits = [m.start() for m in re.finditer(gran, txt)]
    limits.append(len(txt))
    return limits

def extract_main_entity(ent):
    res = ""
    for e in ent.ents:
        if e.label_ == "PERSON":
            res = res + e.text + " "
    return res.strip()

In [36]:
# Timeline for long doucments
# given some text and level of granularity, returns list of entities along with their index
def timeline(txt):
    char_ref = {}
    limits = delimiter(txt)

    st = 0
    res = {}
    for end in limits:
        doc = nlp(txt[st:end])
        for c in doc._.coref_clusters:
            # we only consider proper nouns like 'ALICE'
            main_ent = extract_main_entity(c.main)
                
            if len(main_ent)>0:
                if main_ent not in res:
                    res[main_ent] = []
                #print(str(main_entity), len(c.mentions))
                for ent in c.mentions:
                    #res.append((main_ent, st + ent.start_char))
                    res[main_ent].append((main_ent, st + ent.start_char, st + ent.end_char, ent))
                    char_ref[st + ent.start_char] = main_ent
        st = end + 1
    
    r = []
    for k in res:
        r.extend(res[k])
    
    return r, char_ref

In [37]:
_, char_ref = timeline(book)

In [39]:
# char_ref

In [40]:
char_ref_list = []  # sorted list version of char_ref 
char_freq = {} # freq of each character in the doc. key: char name, value: frequency

for k in sorted(char_ref.keys()):
    ent = char_ref[k]
    char_ref_list.append((k, ent))
    if ent not in char_freq:
        char_freq[ent] = 0
    char_freq[ent] += 1

In [43]:
# char_freq

In [9]:
limits = delimiter(book, gran="para") # index where a 
lim_ind = 0
link_ent = []  # list of lists. Each contituting list contains a set of characters which occur in the same context
temp = []
for (st, ch) in char_ref_list:
    if lim_ind<len(limits) and st>limits[lim_ind]:
        lim_ind += 1
        #print(len(set(temp)))
        link_ent.append(list(set(temp)))
        temp = []
    temp.append(ch)
link_ent.append(list(set(temp)))

In [10]:
char_mapping = {}  # mapping from character to id, Alice -> 1

char_id = 1
for c in char_freq:    
    char_mapping[c] = char_id
    char_id += 1
    
num_char = len(list(char_mapping.keys()))
edges = np.zeros((num_char+1, num_char+1), dtype='int')

for cl in link_ent:
    if len(cl)<=1:
        continue
    for (c1,c2) in itertools.combinations(cl,2):
        edges[char_mapping[c1]][char_mapping[c2]] += 1

In [11]:
# @input
# txt - input text
# char_ref - dictionary key: starting index, value: entity
# output
# char_mapping: dictionary with key: character_name, value:its id
# edges: co-occurence matrix 
def get_graph(txt, char_ref):
    char_ref_list = []  # sorted list version of char_ref 
    char_freq = {} # freq of each character in the doc. key: char name, value: frequency

    for k in sorted(char_ref.keys()):
        ent = char_ref[k]
        char_ref_list.append((k, ent))
        if ent not in char_freq:
            char_freq[ent] = 0
        char_freq[ent] += 1

    limits = delimiter(book, gran="para") # index where a 
    lim_ind = 0
    link_ent = []  # list of lists. Each contituting list contains a set of characters which occur in the same context
    temp = []
    for (st, ch) in char_ref_list:
        if lim_ind<len(limits) and st>limits[lim_ind]:
            lim_ind += 1
            #print(len(set(temp)))
            link_ent.append(list(set(temp)))
            temp = []
        temp.append(ch)
    link_ent.append(list(set(temp)))

    char_mapping = {}  # mapping from character to id, Alice -> 1

    char_id = 1
    for c in char_freq:    
        char_mapping[c] = char_id
        char_id += 1

    num_char = len(list(char_mapping.keys()))
    edges = np.zeros((num_char+1, num_char+1), dtype='int')

    for cl in link_ent:
        if len(cl)<=1:
            continue
        for (c1,c2) in itertools.combinations(cl,2):
            edges[char_mapping[c1]][char_mapping[c2]] += 1
    
    return edges, char_mapping

In [12]:
edges, char_mapping = get_graph(book, char_ref)

In [17]:
edges[13]

array([ 0,  1,  0, 10,  0,  0,  0,  0,  0,  2,  0,  0,  0,  0,  1,  0,  0,
        2,  1,  0,  0,  0,  0,  1,  1,  1,  0,  0,  0,  0,  2,  0,  0,  2,
        3,  0,  1,  0,  1,  1,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,
        0,  1,  0, 18,  0,  1,  3,  1,  1,  2,  1,  0,  0,  1,  0, 13,  0,
        0,  0,  0,  1,  0,  0,  2,  1,  0,  1,  1,  1,  2,  2,  1,  0,  1,
        0,  1, 18,  0,  1,  1,  3,  6,  0,  6,  1,  1,  0,  0,  0,  0,  1,
        0,  1,  0,  0,  1,  0,  0,  1,  0,  1,  3,  1,  1,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  1,  1,  0,  0,  1,  0,  0,  0,  0,  0,  0,
        0,  1,  1,  0,  0,  1,  1,  0,  0,  1,  3,  0,  0,  0,  1,  0,  0,
        1,  0,  0,  0,  0])

In [44]:
# char_mapping

In [45]:
# char_ref

In [25]:
net = Network()
minimum_interaction = 10  # minimum number of interaction to be shown in the graph

# mapping from id to character
id_char_map = {v: k for k, v in char_mapping.items()}
for c1,c2 in np.argwhere(edges>minimum_interaction):
    c1, c2 = c1.item(), c2.item()
    net.add_node(c1, label=id_char_map[c1]) # size=1+math.log(char_freq[c], 10)
    net.add_node(c2, label=id_char_map[c2])
    net.add_edge(c1, c2)
    
net.show('example.html')

In [28]:
net = Network()
minimum_interaction = 5  # minimum number of interaction to be shown in the graph

# mapping from id to character
id_char_map = {v: k for k, v in char_mapping.items()}
for c1 in np.argwhere(edges[13]>minimum_interaction):
    c1 = c1.item()
    c2 = 13
    net.add_node(c1, label=id_char_map[c1]) # size=1+math.log(char_freq[c], 10)
    net.add_node(c2, label=id_char_map[c2])
    net.add_edge(c1, c2)
    
net.show('example.html')

### graph naimul version

In [64]:
def extract_main_entity(ent):
    res = ""
    for e in ent.ents:
        if e.label_ == "PERSON":
            res = res + e.text + " "
    return res.strip().replace("'d","").replace(".","").replace("'","")

def split_by_paragraphs(txt, max_para = 1):
    limits = []
    for i, m in enumerate(re.finditer('\n\n', txt)):
        if i>0 and i%max_para==0: 
            limits.append(m.start())
    limits.append(len(txt))
    
    return limits

In [65]:
'''
given some text and level of granularity, returns list of entities along with their index
'''

txt = book
user_defined = []

splits = split_by_paragraphs(txt)

# print("user defined:", user_defined)

### iterace the splits and calculate character mentions
st = 0
docs = []
sent_i = 0
entities = {}
res = []
mapping = []
full_mapping = []
char_ref = {}
ind = 0

for end in splits:
    doc = nlp(txt[st:end])
    docs.append(doc)

    ## collect user defined entities
    i = 0
    mapping = []
    for sent in doc.sents:
        mapping.append(sent.start_char+st)
        for c in user_defined:
            if ' '+ c + ' ' in sent.text:
                print("enter")
                res.append({"name":c, "sentence":i+sent_i, "index": i+sent_i})
                entities[c] = 1
        i += 1
    # print("mapping", mapping)
    # mapping = [sent.start_char+st for sent in doc.sents]
    full_mapping.extend(mapping)
    for c in doc._.coref_clusters:
        main_ent = extract_main_entity(c.main)
        if len(main_ent)>0:
            for ent in c.mentions:
                for i in range(len(mapping)):
                    m = mapping[i]
                    if ent.start_char+st > m:
                        ind = i+sent_i
                res.append({"name":main_ent, "sentence":ind, "index": ind})
                char_ref[st + ent.start_char] = main_ent
                entities[main_ent] = 1

    sent_i += len(mapping)
    st = end + 1

In [69]:
st = 0
sent_i = 0
edges = []
for i in range(len(splits)):
    end = splits[i]
    doc = docs[i]
    
    for sent in doc.sents:
#         print(sent)
        edges.append(set())
        for t in sent:
            if st+t.idx in char_ref:
                ent = char_ref[st + t.idx]
#                 print(ent)
                edges[sent_i].add(ent)
                
        sent_i += 1
                
    st = end + 1

In [70]:
res = []
unique_chars = dict()
for entry in edges:
    if len(entry) > 1:
        for (c1,c2) in itertools.combinations(entry,2):
            res.append({"source": c1, "target": c2, "value": 1})
            unique_chars[c1] = 1
            unique_chars[c2] = 1

In [71]:
res

[{'source': 'Dursley', 'target': 'Dudley', 'value': 1},
 {'source': 'Dursley', 'target': 'Dudley', 'value': 1},
 {'source': 'Dumbledore', 'target': 'McGonagall', 'value': 1},
 {'source': 'Dumbledore', 'target': 'Harry', 'value': 1},
 {'source': 'Dudley', 'target': 'Harry', 'value': 1},
 {'source': 'Dudley', 'target': 'Harry', 'value': 1},
 {'source': 'Dudley', 'target': 'Harry', 'value': 1},
 {'source': 'Dudley', 'target': 'Harry', 'value': 1},
 {'source': 'Uncle Vernon', 'target': 'Harry', 'value': 1},
 {'source': 'Dudley', 'target': 'Harry', 'value': 1},
 {'source': 'Dudley', 'target': 'Harry', 'value': 1},
 {'source': 'Dudley', 'target': 'Atta Dudley', 'value': 1},
 {'source': 'Dudley', 'target': 'Atta Dudley', 'value': 1},
 {'source': 'Figg', 'target': 'Harry', 'value': 1},
 {'source': 'Aunt Petunia', 'target': 'Harry', 'value': 1},
 {'source': 'Aunt Petunia', 'target': 'Harry', 'value': 1},
 {'source': 'Dudley', 'target': 'Harry', 'value': 1},
 {'source': 'Dudley', 'target': 'Harr

In [68]:
unique_chars

{'Kate Warren': 1,
 'Tony Johnson': 1,
 'Chris': 1,
 'Kate': 1,
 'McDonough': 1,
 'Jake': 1,
 'Gus': 1}

In [75]:
txt = '''Once upon a time there lived a King and a Queen, who lacked but one thing on earth to make them entirely happy. The King was young, handsome, and wealthy; the Queen had a nature as good and gentle as her face was beautiful; and they adored one another, having married for love—which among kings and queens is not always the rule. Moreover, they reigned over a kingdom at peace, and their people were devoted to them. What more, then, could they possibly want? Well, they wanted one thing very badly, and the lack of it grieved them more than words can tell. They had no child. Vows, pilgrimages, all ways were tried; yet for a long while nothing came of it all, and the poor Queen especially was in despair. At last, however, to her own and her husband's inexpressible joy, she gave birth to a daughter. As soon as the palace guns announced this event, the whole nation went wild with delight. Flags waved everywhere, bells were set pealing until the steeples rocked, crowds tossed up their hats and cheered, while the soldiers presented arms, and even strangers meeting in the street fell upon each other's neck, exclaiming: 'Our Queen has a daughter! Yes, yes—Our Queen has a daughter! Long live the little Princess!' A name had now to be found for the royal babe; and the King and Queen, after talking over some scores of names, at length decided to call her Aurora, which means The Dawn. The Dawn itself (thought they) was never more beautiful than this darling of theirs. The next business, of course, was to hold a christening. They agreed that it must be a magnificent one; and as a first step they invited all the Fairies they could find in the land to be godmothers to the Princess Aurora, that each one of them might bring her a gift, as was the custom with Fairies in those days, and so she might have all the perfections imaginable. After making long inquiries—for I should tell you that all this happened not so many hundred years ago, when Fairies were already growing somewhat scarce—they found seven. But this again pleased them, because seven is a lucky number. After the ceremonies of the christening, while the trumpeters sounded their fanfares and the guns boomed out again from the great tower, all the company returned to the Royal Palace to find a great feast arrayed. Seats of honour had been set for the seven fairy godmothers, and before each was laid a dish of honour, with a dish-cover of solid gold, and beside the dish a spoon, a knife, and a fork, all of pure gold and all set with diamonds and rubies. But just as they were seating themselves at table, to the dismay of every one there appeared in the doorway an old crone, dressed in black and leaning on a crutched stick. Her chin and her hooked nose almost met together, like a pair of nut-crackers, for she had very few teeth remaining; but between them she growled to the guests in a terrible voice:' I am the Fairy Uglyane! Pray where are your King's manners, that I have not been invited?' She had in fact been overlooked; and this was not surprising, because she lived at the far end of the country, in a lonely tower set around by the forest. For fifty years she had never come out of this tower, and every one believed her to be dead or enchanted. That, you must know, is the commonest way the Fairies have of ending: they lock themselves up in a tower or within a hollow oak, and are never seen again.

The King, though she chose to accuse his manners, was in fact the politest of men. He hurried to express his regrets, led her to table with his own hand, and ordered a dish to be set for her; but with the best will in the world he could not give her a dish-cover such as the others had, because seven only had been made for the seven invited Fairies. The old crone received his excuses very ungraciously, while accepting a seat. It was plain that she had taken deep offence. One of the younger Fairies, Hippolyta by name, who sat by, overheard her mumbling threats between her teeth; and fearing she might bestow some unlucky gift upon the little Princess, went as soon as she rose from table and hid herself close by the cradle, behind the tapestry, that she might have the last word and undo, so far as she could, what evil the Fairy Uglyane might have in her mind.

She had scarcely concealed herself before the other Fairies began to advance, one by one, to bestow their gifts on the Princess. The youngest promised her that she should be the most beautiful creature in the world; the next, that she should have the wit of an angel; the third, a marvellous grace in all her ways; the fourth, that she should dance to perfection; the fifth, that she should sing like a nightingale; the sixth, that she should play exquisitely on all instruments of music.

Now came the turn of the old Fairy Uglyane. Her head nodded with spite and old age together, as she bent over the cradle and shook her crutched staff above the head of the pretty babe, who slept on sweetly, too young and too innocent as yet to dream of any such thing as mischief in this world.

'This is my gift to you, Princess Aurora,' announced the hag, still in her creaking voice that shook as spitefully as her body. 'I promise that one day you shall pierce your hand with a spindle, and on that day you shall surely die!' At these terrible words the poor Queen fell back fainting into her husband's arms. A trembling seized the whole Court; the ladies were in tears, and the younger lords and knights were calling out to seize and burn the wicked witch, when the young Fairy stepped forth from behind the tapestry, and passing by Uglyane, who stood scornful in the midst of this outcry, she thus addressed their Majesties: 'Take comfort, O King and Queen: your daughter shall not die thus. It is true, I have not the power wholly to undo what this elder sister of mine has done. The Princess must indeed pierce her hand with a spindle; but, instead of dying, she shall only fall into a deep slumber that shall last for many, many years, at the end of which a King's son shall come and awake her. Whenever this misfortune happens to your little Aurora, do not doubt that I, the Fairy Hippolyta, her godmother, shall get news of it and come at once to render what help I may.'



The King, while declaring himself infinitely obliged to the good Fairy Hippolyta, could not help feeling that hers was but cold comfort at the best. He gave orders to close the christening festivities at once, although the Fairy Uglyane, their spoil-joy, had already taken her departure; passing unharmed through the crowd of folk, every one of whom wished her ill, and riding away—it was generally agreed-upon a broomstick.

To satisfy the King's faithful subjects, however,—who were unaware of any misadventure—the palace fireworks were duly let off, with a grand set-piece wishing Long Life to the Princess Aurora! in all the colours of the rainbow. But His Majesty, after bowing from the balcony amid the banging of rockets and hissing of Catherine wheels, retired to a private room with his Chamberlain, and there, still amid the noise of explosions and cheering, drew up the first harsh proclamation of his reign. It forbade every one, on pain of death, to use a spindle in spinning or even to have a spindle in his house. Heralds took copies of this proclamation and marched through the land reading it, to the sound of trumpets, from every market-place: and it gravely 

[Pg 8]

puzzled and distressed all who listened, for their women folk prided themselves on their linen. Its fineness was a byword throughout the neighbouring kingdoms, and they knew themselves to be famous for it. 'But what sort of linen,' said they, 'would His Majesty have us spin without spindles?'

They had a great affection, however (as we have seen), for their monarch; and for fifteen or sixteen years all the spinning-wheels were silent throughout the land. The little Princess Aurora grew up without ever having seen one. But one day—the King and Queen being absent at one of their country houses—she gave her governess the slip, and running at will through the palace and upstairs from one chamber to another, she came at length to a turret with a winding staircase, from the top of which a strange whirring sound attracted her and seemed to invite her to climb. As she mounted after the sound, on a sudden it ceased; but still she followed the stairs and came, at the very top, to an open door through which she looked in upon a small garret where sat an honest old woman alone, winding her distaff. The good soul had never, in sixteen years, heard of the King's prohibition against spindles; 

[Pg 9]

and this is just the sort of thing that happens in palaces.

'What are you doing, goody?' asked the Princess.

'I am spinning, pretty one,' answered the old woman, who did not know who she was.

'Spinning? What is that?'

'I wonder sometimes,' said the old woman, 'what the world is coming to, in these days!' And that, of course, was natural enough, and might occur to anybody after living so long as she had lived in a garret on the top of a tower. 'Spinning,' she said wisely, 'is spinning, or was; and, gentle or simple, no one is fit to keep house until she has learnt to spin.'

'But how pretty it is!' said the Princess. 'How do you do it? Give it to me and let me see if I can do so well.'

She had no sooner grasped the spindle—she was over-eager perhaps, or just a little bit clumsy, or maybe the fairy decree had so ordained it—than it pierced her hand and she dropped down in a swoon.

The old trot in a flurry ran to the head of the 

[Pg 10]

stairs and called for help. There was no bell rope, and, her voice being weak with age and her turret in the remotest corner of the palace, it was long before any one heard her in the servants' hall. The servants, too—in the absence of the King and Queen—were playing cards, and could not be interrupted by anybody until their game was finished. Then they sat down and discussed whose business it was to attend on a call from that particular turret; and this again proved to be a nice point, since nobody could remember having been summoned thither, and all were against setting up a precedent (as they called it). In the end they decided to send up the lowest of the junior page-boys. But he had a weakness which he somehow forgot to mention—that of fainting at the sight of blood. So when he reached the garret and fainted, the old woman had to begin screaming over again.

This time they sent up a scullery maid; who, being good-natured and unused to the ways of the palace, made the best haste she could to the garret, whence presently she returned with the terrible news. The servants, who had gone back to their game, now dropped their cards and came running. 

[Pg 11]

All the household, in fact, came pouring up the turret stairs; the palace physicians themselves crowding in such numbers that the poor Princess Aurora would have been hard put to it for fresh air could fresh air have restored her. They dashed water on her face, unlaced her, slapped her hands, tickled the soles of her feet, burned feathers under her nose, rubbed her temples with Hungary-water. They held consultations over her, by twos and threes, and again in Grand Committee. But nothing would bring her to.

Meanwhile, a messenger had ridden off posthaste with the tidings, and while the doctors were still consulting and shaking their heads the King himself came galloping home to the palace. In the midst of his grief he bethought him of what the Fairies had foretold; and being persuaded that, since they had said it, this was fated to happen, he blamed no one but gave orders to carry the Princess to the finest apartment in the palace, and there lay her on a bed embroidered with gold and silver.

At sight of her, she was so lovely, you might well have supposed that some bright being of the skies had floated down to earth and there dropped asleep after her long journey. For her swoon had not taken away the warm tints of her complexion: her cheeks were like carnations, her lips like coral: and though her eyes were closed and the long lashes would not lift, her soft breathing told that she was not dead. The King commanded them all to leave her and let her sleep in peace until the hour of her awakening should arrive.

Now when the accident befell our Princess the good Fairy Hippolyta, who had saved her life, happened to be in the Kingdom of Mataquin, twelve thousand leagues away; but news of it was brought to her in an incredibly short space of time by a little dwarf who owned a pair of seven-league boots. (These were boots in which you could walk seven leagues at a single stride.) She set off at once to the help of her beloved goddaughter, and behold in an hour this good Fairy arrived at the palace, in a fiery chariot drawn by dragons.

Our King met her and handed her down from the chariot. She approved of all that he had done; but, greatly foreseeing as she was, she bethought her that, as all mortals perish within a hundred years or so, when the time came for the Princess to awake she would be distressed at finding herself orphaned and alone in this old castle.

So this is what she did. She touched with her wand everything and everybody in the palace: the King, the Queen; the ministers and privy councillors; the archbishop (who was the Grand Almoner), the bishops and the minor clergy; the maids-of-honour, ladies of the bedchamber, governesses, gentlemen-in-waiting, equerries, heralds, physicians, officers, masters of the household, cooks, scullions, lackeys, guards, Switzers, pages, footmen. She touched the Princess's tutors and the Court professors in the midst of their deep studies. She touched likewise all the horses in the stables, with the grooms; the huge mastiffs in the yard; even Tiny, the Princess's little pet dog, and Fluff, her black-and-white cat, that lay coiled on a cushion by her bedside.

The instant the Fairy Hippolyta touched them they all fell asleep, not to awake until the same moment as their mistress, that all might be ready to wait on her when she needed them. The very spits at the fire went to sleep, loaded as they were with partridges and pheasants; and the fire went to sleep too. All this was done in a moment: the Fairies were never long about their business in those days.

But it so happened that one of the King's councillors, the Minister of Marine (his office dated from a previous reign when the kingdom had hoped to conquer and acquire a seaboard) had overslept himself that morning and came late to the palace without any knowledge of what had befallen. He felt no great fear that his unpunctuality would be remarked, the King (as he supposed) being absent in the country; nevertheless he took the precaution of letting himself in by a small postern door, and so missed being observed by the Fairy and touched by her wand. Entering his office, and perceiving that his under-secretary (usually so brisk) and all his clerks rested their heads on their desks in attitudes of sleep, he drew the conclusion that something had happened, for he was an excellent judge of natural slumber. The farther he penetrated into the palace, the stronger his suspicions became. He withdrew on tiptoe. Though by nature and habit a lazy man, he was capable of sudden decision, and returning to his home he caused notices to be posted up, forbidding any one to approach the castle, the inmates of which were suffering from an Eastern but temporary affliction known as the Sleeping Sickness. These notices were unnecessary, for within a few hours there grew up, all around the park, such a number of trees of all sizes, and such a tangle of briars and undergrowth, that neither beast nor man could find a passage. They grew until nothing but the tops of the castle towers could be seen, and these only from a good way off. There was no mistake about it: the Fairy had done her work well, and the Princess might sleep with no fear of visits from the inquisitive.

One day, many, many years afterwards, the incomparable young Prince Florimond happened to ride a-hunting on that side of the country which lay next to the tangled forest, and asked: 'What were those towers he saw pushing up above the midst of a great thick wood? '

They all answered him as they heard tell. Some said it was an old castle haunted by ghosts.

Others, that all the wizards and witches of the country met there to keep Sabbath.

The most general opinion was that an Ogre dwelt there, and that he carried off thither all the children he could catch, to eat them at his ease. No one could follow him, for he alone knew how to find a passage through the briars and brambles. The Prince could not tell which to believe of all these informants, for all gave their versions with equal confidence, as commonly happens with those who talk on matters of which they can know nothing for certain. He was turning from one to another in perplexity, when a peasant spoke up and said:—

'Your Highness, long ago I heard my father tell that there was in yonder castle a Princess, the most beautiful that ever man saw; that she must lie asleep there for many, many years; and that one day she will be awakened by a King's son, for whom she was destined.'

At these words Prince Florimond felt himself a-fire. He believed, without weighing it, that he could accomplish this fine adventure; and, spurred on by love and ambition, he resolved to explore then and there and discover the truth for himself.

'''

In [76]:
with open('../static/data/story/sleeping_beauty.txt', 'w') as f:
    f.write(txt)

In [74]:
f.close()

In [77]:
sample = 'Mr. Darling was frightfully proud of her, but he was very honourable, and he sat on the edge of Mrs. Darling’s bed, holding her hand and calculating expenses, while she looked at him imploringly.'

In [78]:
doc = nlp(sample)

In [86]:
for t in doc:
    if t.dep_ == 'acomp':
        print(t, t.dep_)
        for c in t.head.children:
            print(c, c.idx)

proud acomp
Darling 4
proud 28
, 40
but 42
was 49
honourable acomp
he 46
honourable 58
, 68
and 70
sat 77
